In [1]:
import os
import sys
sys.path.append('/Users/nvalett/Documents/Natalie/Species Dist Research/Thesis/EcoLinker')
from EcoLinker.restorationOptimizer import restorationOptimizer
import ecoscape_connectivity
from scgt import Tile, GeoTiff
import numpy as np

DATA_PATH="/Users/nvalett/Documents/Natalie/Species Dist Research/Thesis/EcoLinker/tests/assets/paper_repro"

HABITAT_PATH = os.path.join(DATA_PATH, "habitat.tif")
TERRAIN_PATH = os.path.join(DATA_PATH, "terrain.tif")
PERMEABILITY_PATH = os.path.join(DATA_PATH, "transmission_refined_1.csv")
transmission_d = ecoscape_connectivity.util.read_transmission_csv(PERMEABILITY_PATH)

CONNECTIVITY_PATH = os.path.join(DATA_PATH, "repop.tif")
FLOW_PATH = os.path.join(DATA_PATH, "grad.tif")

optimizer = restorationOptimizer(HABITAT_PATH, TERRAIN_PATH, CONNECTIVITY_PATH, FLOW_PATH, PERMEABILITY_PATH, 10)

In [2]:
death_tif = optimizer.get_death_layer("death.tif")
print("death:")
# death_tif.draw_geotiff()

death:


In [3]:
highest_death = optimizer.get_highest_death_pixels(death_tif)
print(highest_death)

print(optimizer.get_most_permiable_terrain())

print('Restoration:')
print(f'Pre-restoration sum of connectivity {optimizer.sum_of_connectivity()}')

for x, y in highest_death.keys():
    with GeoTiff.from_file(optimizer.terrain_fn) as terrain_geotiff:
        old_permiability = terrain_geotiff.get_pixel_value(x, y)

    optimizer.change_terrain(x, y)

    with GeoTiff.from_file(optimizer.terrain_fn) as terrain_geotiff:
        new_permiability = terrain_geotiff.get_pixel_value(x, y)
    print(f'Restoring pixel ({x}, {y}) from permiability {optimizer.permeability_dict[old_permiability]} to {optimizer.permeability_dict[new_permiability]}')

{(1161, 3335): 60, (862, 2169): 58, (859, 2160): 57, (864, 2173): 56, (864, 2174): 56, (1441, 2115): 56, (864, 2172): 56, (865, 2173): 55, (993, 1858): 55}
104
Restoration:
Pre-restoration sum of connectivity 174190808
Restoring pixel (1161, 3335) from permiability 0.0009343065693430657 to 1.0
Restoring pixel (862, 2169) from permiability 0.031532846715328466 to 1.0
Restoring pixel (859, 2160) from permiability 0.031532846715328466 to 1.0
Restoring pixel (864, 2173) from permiability 0.031532846715328466 to 1.0
Restoring pixel (864, 2174) from permiability 0.031532846715328466 to 1.0
Restoring pixel (1441, 2115) from permiability 0.031532846715328466 to 1.0
Restoring pixel (864, 2172) from permiability 0.031532846715328466 to 1.0
Restoring pixel (865, 2173) from permiability 0.031532846715328466 to 1.0
Restoring pixel (993, 1858) from permiability 0.04525547445255475 to 1.0


In [4]:
# compute connectivity - not needed unless output is needed
ecoscape_connectivity.compute_connectivity(HABITAT_PATH, TERRAIN_PATH, 'repop_after_restoration.tif', None, transmission_d)
print(f'post-restoration sum of connectivity {optimizer.sum_of_connectivity()}')